# 03 - Building a Fictional Neighbourhood

This notebook generates 100 synthetic personas for the Testing space before it happens experiment.

**Scenario:** 42 households surrounding a 0.3-acre plot in Dalston, East London (E8) have formed a Community Land Trust. We need to simulate ~100 residents with realistic demographics and household structures.

**Pipeline:**
1. Define target demographics based on E8 census data
2. Generate street layout and addresses
3. Allocate household types to addresses
4. Generate people within each household
5. Add individuation (personality, attitudes, data files)
6. Validate and export

In [1]:
import sys
sys.path.insert(0, '../src')

from pathlib import Path
import json
import random
from dataclasses import dataclass, field, asdict
from typing import Optional
import uuid

from tqdm.auto import tqdm

from centuria.llm import complete, estimate_cost, CostEstimate
from centuria.persona import create_persona, generate_synthetic_files, SyntheticIdentity
from centuria.data import process_personal_folder

---
## Part 1: Demographics Definition

Define target distributions based on E8 Dalston demographics. These are approximations informed by census data and the area's character as a gentrifying inner-London neighbourhood.

In [2]:
# E8 Dalston Target Demographics
# Based on 2021 Census and local knowledge of gentrification patterns

DEMOGRAPHICS = {
    # Household composition (must sum to 1.0)
    "household_types": {
        "single_person": 0.30,
        "couple_no_children": 0.20,
        "family_with_children": 0.25,
        "house_share": 0.15,
        "multi_generational": 0.10,
    },
    
    # Tenure distribution
    "tenure": {
        "owner_occupied": 0.35,
        "private_rented": 0.35,
        "social_housing": 0.30,
    },
    
    # Ethnicity (E8 is highly diverse)
    "ethnicity": {
        "white_british": 0.30,
        "white_other": 0.15,  # European migrants
        "black_british_caribbean": 0.12,
        "black_british_african": 0.10,
        "turkish_kurdish": 0.12,  # Large local community
        "south_asian": 0.08,
        "east_asian": 0.05,
        "mixed": 0.08,
    },
    
    # Age distribution (skews younger than UK average)
    "age_brackets": {
        "18-24": 0.12,
        "25-34": 0.28,
        "35-44": 0.22,
        "45-54": 0.15,
        "55-64": 0.12,
        "65+": 0.11,
    },
    
    # Socioeconomic (reflects gentrification mix)
    "socioeconomic": {
        "working_class": 0.35,
        "lower_middle": 0.25,
        "middle_class": 0.25,
        "professional": 0.15,
    },
    
    # Years in area (gentrification creates bimodal distribution)
    "years_in_area": {
        "0-2": 0.20,   # Recent arrivals
        "3-5": 0.20,
        "6-10": 0.15,
        "11-20": 0.20,
        "20+": 0.25,   # Long-term residents
    },
}

# Validate distributions sum to 1.0
for category, distribution in DEMOGRAPHICS.items():
    total = sum(distribution.values())
    assert abs(total - 1.0) < 0.01, f"{category} sums to {total}, not 1.0"
    
print("Demographics validated. Target distributions:")
for category, distribution in DEMOGRAPHICS.items():
    print(f"\n{category}:")
    for key, value in distribution.items():
        print(f"  {key}: {value:.0%}")

Demographics validated. Target distributions:

household_types:
  single_person: 30%
  couple_no_children: 20%
  family_with_children: 25%
  house_share: 15%
  multi_generational: 10%

tenure:
  owner_occupied: 35%
  private_rented: 35%
  social_housing: 30%

ethnicity:
  white_british: 30%
  white_other: 15%
  black_british_caribbean: 12%
  black_british_african: 10%
  turkish_kurdish: 12%
  south_asian: 8%
  east_asian: 5%
  mixed: 8%

age_brackets:
  18-24: 12%
  25-34: 28%
  35-44: 22%
  45-54: 15%
  55-64: 12%
  65+: 11%

socioeconomic:
  working_class: 35%
  lower_middle: 25%
  middle_class: 25%
  professional: 15%

years_in_area:
  0-2: 20%
  3-5: 20%
  6-10: 15%
  11-20: 20%
  20+: 25%


---
## Data Structures

Define the core data classes for addresses, households, and people.

In [3]:
TARGET_POPULATION = 100  # Exact number of people to generate

@dataclass
class Address:
    street: str
    number: str
    postcode: str = "E8 2PB"
    property_type: str = "Victorian terrace"  # or "converted_flat", "council_flat", "new_build"
    
    def __str__(self):
        return f"{self.number} {self.street}, London {self.postcode}"


@dataclass
class HouseholdMember:
    """Member of a household - fields match SyntheticIdentity for easy conversion."""
    id: str
    name: str
    age: int
    gender: str
    relationship: str  # "head", "spouse", "partner", "child", "parent", "flatmate"
    occupation: str
    industry: str  # Industry sector for SyntheticIdentity
    education: str  # Education level for SyntheticIdentity
    ethnicity: str
    political_lean: str
    personality_sketch: str
    

@dataclass
class Household:
    id: str
    address: Address
    household_type: str
    tenure: str
    years_in_area: int
    socioeconomic: str
    target_size: int = 1  # Pre-calculated to ensure total = TARGET_POPULATION
    members: list[HouseholdMember] = field(default_factory=list)
    
    @property
    def size(self) -> int:
        return len(self.members)


@dataclass
class Neighbourhood:
    name: str
    location: str
    households: list[Household] = field(default_factory=list)
    
    @property
    def total_residents(self) -> int:
        return sum(h.size for h in self.households)
    
    def all_members(self) -> list[HouseholdMember]:
        return [m for h in self.households for m in h.members]
    
    def get_member_by_id(self, member_id: str) -> Optional[HouseholdMember]:
        for h in self.households:
            for m in h.members:
                if m.id == member_id:
                    return m
        return None

print(f"Data structures defined. Target population: {TARGET_POPULATION}")
print(f"HouseholdMember now includes: industry, education (for SyntheticIdentity compatibility)")

Data structures defined. Target population: 100
HouseholdMember now includes: industry, education (for SyntheticIdentity compatibility)


---
## Stage 1: Streets & Addresses

Generate 42 addresses on fictional streets surrounding the plot. The streets should feel authentically Dalston - Victorian terraces mixed with post-war council blocks and some newer developments.

In [4]:
# CACHING: Check if neighbourhood data already exists
NEIGHBOURHOOD_FILE = Path('../data/synthetic/dalston_clt/neighbourhood.json')
PERSONAS_FILE = Path('../data/synthetic/dalston_clt/personas.json')

if NEIGHBOURHOOD_FILE.exists() and PERSONAS_FILE.exists():
    print("=" * 60)
    print("CACHED DATA FOUND - Loading existing neighbourhood")
    print("=" * 60)
    print(f"\n  {NEIGHBOURHOOD_FILE}")
    print(f"  {PERSONAS_FILE}")
    print("\n  To regenerate, delete these files and rerun.")
    
    SKIP_GENERATION = True
else:
    print("No cached data found. Will generate new neighbourhood.")
    SKIP_GENERATION = False

if not SKIP_GENERATION:
    # Generate new street names
    STREETS_PROMPT = """Generate 4 fictional street names for a neighbourhood in Dalston, East London (E8).

    The streets should:
    - Sound authentically London (mix of historical figures, local landmarks, trees, or trades)
    - NOT be real Dalston street names
    - Reflect the area's character (Victorian working-class origins, now gentrifying)

    Return JSON array of 4 street names only, no "Street" or "Road" suffix:
    ["Name1", "Name2", "Name3", "Name4"]

    Return ONLY the JSON array."""

    result = await complete(STREETS_PROMPT)
    text = result.content.strip()
    if text.startswith("```"): 
        text = text.split("```")[1].replace("json", "", 1).strip()
    street_names = json.loads(text)

    # Define street types and property mix
    STREETS = [
        {"name": f"{street_names[0]} Road", "numbers": range(1, 25, 2), "property_type": "Victorian terrace"},
        {"name": f"{street_names[1]} Street", "numbers": range(2, 20, 2), "property_type": "Victorian terrace"},
        {"name": f"{street_names[2]} Estate", "numbers": [f"Flat {i}" for i in range(1, 13)], "property_type": "Council flat"},
        {"name": f"{street_names[3]} Mews", "numbers": range(1, 9), "property_type": "New build"},
    ]

    print("Generated streets:")
    for street in STREETS:
        num_properties = len(list(street["numbers"]))
        print(f"  {street['name']}: {num_properties} properties ({street['property_type']})")
else:
    print("\n✓ Skipping street generation (using cached data)")

CACHED DATA FOUND - Loading existing neighbourhood

  ../data/synthetic/dalston_clt/neighbourhood.json
  ../data/synthetic/dalston_clt/personas.json

  To regenerate, delete these files and rerun.

✓ Skipping street generation (using cached data)


In [5]:
if not SKIP_GENERATION:
    # Generate 42 addresses distributed across streets
    addresses = []

    for street in STREETS:
        for number in street["numbers"]:
            addresses.append(Address(
                street=street["name"],
                number=str(number),
                property_type=street["property_type"]
            ))
            if len(addresses) >= 42:
                break
        if len(addresses) >= 42:
            break

    print(f"Generated {len(addresses)} addresses:")
    for addr in addresses[:5]:
        print(f"  {addr} ({addr.property_type})")
    print(f"  ... and {len(addresses) - 5} more")
else:
    print("✓ Skipping address generation (using cached data)")

✓ Skipping address generation (using cached data)


---
## Stage 2: Household Allocation

Assign household characteristics to each address based on target demographics. Property type influences likely tenure and household type.

In [6]:
def weighted_choice(distribution: dict) -> str:
    """Select from distribution based on weights."""
    items = list(distribution.keys())
    weights = list(distribution.values())
    return random.choices(items, weights=weights, k=1)[0]


def years_from_bracket(bracket: str) -> int:
    """Convert bracket like '6-10' to a random year in that range."""
    if bracket == "20+":
        return random.randint(20, 45)
    parts = bracket.split("-")
    return random.randint(int(parts[0]), int(parts[1]))


def get_expected_household_size(household_type: str) -> tuple[int, int]:
    """Return (min, max) household size for type."""
    sizes = {
        "single_person": (1, 1),
        "couple_no_children": (2, 2),
        "family_with_children": (3, 5),
        "house_share": (2, 4),
        "multi_generational": (3, 6),
    }
    return sizes.get(household_type, (1, 4))


# Property type influences tenure distribution
TENURE_BY_PROPERTY = {
    "Victorian terrace": {"owner_occupied": 0.45, "private_rented": 0.45, "social_housing": 0.10},
    "Council flat": {"owner_occupied": 0.10, "private_rented": 0.15, "social_housing": 0.75},
    "New build": {"owner_occupied": 0.50, "private_rented": 0.45, "social_housing": 0.05},
}

print("Helper functions defined.")

Helper functions defined.


In [7]:
if not SKIP_GENERATION:
    # Create households with allocated characteristics
    households = []

    for i, address in enumerate(addresses):
        # Get tenure based on property type
        tenure_dist = TENURE_BY_PROPERTY.get(address.property_type, DEMOGRAPHICS["tenure"])
        tenure = weighted_choice(tenure_dist)
        
        # Other characteristics from general demographics
        household_type = weighted_choice(DEMOGRAPHICS["household_types"])
        socioeconomic = weighted_choice(DEMOGRAPHICS["socioeconomic"])
        years_bracket = weighted_choice(DEMOGRAPHICS["years_in_area"])
        years = years_from_bracket(years_bracket)
        
        # Initial size based on household type (will be adjusted to hit target)
        min_size, max_size = get_expected_household_size(household_type)
        initial_size = random.randint(min_size, max_size)
        
        household = Household(
            id=f"hh_{i+1:03d}",
            address=address,
            household_type=household_type,
            tenure=tenure,
            years_in_area=years,
            socioeconomic=socioeconomic,
            target_size=initial_size,
            members=[]
        )
        households.append(household)

    # Adjust household sizes to sum to exactly TARGET_POPULATION
    current_total = sum(h.target_size for h in households)
    print(f"Initial allocation: {current_total} people across {len(households)} households")
    print(f"Adjusting by {TARGET_POPULATION - current_total:+d} to reach {TARGET_POPULATION}...")

    # Keep adjusting until we hit the target
    max_iterations = 100
    iteration = 0

    while sum(h.target_size for h in households) != TARGET_POPULATION and iteration < max_iterations:
        current_total = sum(h.target_size for h in households)
        difference = TARGET_POPULATION - current_total
        
        if difference > 0:
            adjustable = [h for h in households if h.target_size < get_expected_household_size(h.household_type)[1]]
            if not adjustable:
                adjustable = [h for h in households if h.household_type != "single_person"]
            if not adjustable:
                adjustable = list(households)
            random.shuffle(adjustable)
            adjustable[0].target_size += 1
            
        elif difference < 0:
            adjustable = [h for h in households if h.target_size > get_expected_household_size(h.household_type)[0]]
            if not adjustable:
                adjustable = [h for h in households if h.target_size > 1]
            if adjustable:
                random.shuffle(adjustable)
                adjustable[0].target_size -= 1
        
        iteration += 1

    final_total = sum(h.target_size for h in households)
    assert final_total == TARGET_POPULATION, f"Failed to reach target after {max_iterations} iterations: got {final_total}"

    print(f"Final allocation: {final_total} people across {len(households)} households")
    print(f"Average household size: {final_total / len(households):.2f}\n")

    # Show distribution summary
    from collections import Counter

    print("Household type distribution:")
    type_counts = Counter(h.household_type for h in households)
    for htype, count in sorted(type_counts.items()):
        print(f"  {htype}: {count} ({count/len(households):.0%})")

    print("\nTenure distribution:")
    tenure_counts = Counter(h.tenure for h in households)
    for tenure, count in sorted(tenure_counts.items()):
        print(f"  {tenure}: {count} ({count/len(households):.0%})")

    print("\nHousehold size distribution:")
    size_counts = Counter(h.target_size for h in households)
    for size, count in sorted(size_counts.items()):
        print(f"  {size} person(s): {count} households")
else:
    print("✓ Skipping household allocation (using cached data)")

✓ Skipping household allocation (using cached data)


---
## Stage 3: Generate People

For each household, generate members using the LLM. Pass context about the household and a running list of already-generated names to prevent duplication.

In [8]:
def build_household_generation_prompt(household: Household, existing_names: list[str]) -> str:
    """Build prompt for generating household members. Uses household.target_size."""
    
    target_size = household.target_size
    
    # Build household context - adjust description based on actual target size
    if household.household_type == "single_person":
        household_desc = "a single adult living alone"
    elif household.household_type == "couple_no_children":
        household_desc = "a couple without children"
    elif household.household_type == "family_with_children":
        num_children = max(1, target_size - 2)
        household_desc = f"a family with {num_children} child(ren)"
    elif household.household_type == "house_share":
        household_desc = f"{target_size} unrelated adults sharing the property"
    elif household.household_type == "multi_generational":
        household_desc = "multiple generations living together (e.g., grandparent, parent, child)"
    else:
        household_desc = f"{target_size} people"
    
    tenure_desc = {
        "owner_occupied": "owns the property",
        "private_rented": "rents privately",
        "social_housing": "lives in social/council housing",
    }
    
    socio_desc = {
        "working_class": "working-class background (manual labour, service jobs, trades)",
        "lower_middle": "lower-middle class (clerical, small business, skilled trades)",
        "middle_class": "middle class (teachers, nurses, mid-level professionals)",
        "professional": "professional class (lawyers, doctors, tech workers, executives)",
    }
    
    existing_names_str = ", ".join(existing_names[-50:]) if existing_names else "None yet"
    
    prompt = f"""Generate EXACTLY {target_size} person(s) for a household in Dalston, East London (E8).

HOUSEHOLD CONTEXT:
- Address: {household.address}
- Type: {household_desc}
- Tenure: {tenure_desc[household.tenure]}
- Socioeconomic: {socio_desc[household.socioeconomic]}
- Years in area: {household.years_in_area}
- Property: {household.address.property_type}

DALSTON DEMOGRAPHICS:
- Highly diverse: White British 30%, Black British 22%, Turkish/Kurdish 12%, South Asian 8%, White European 15%, Mixed 8%, East Asian 5%
- This household's ethnicity should be consistent (family members share ethnicity, flatmates may differ)
- Names should reflect ethnicity realistically
- Include young adults (18-24) and older people (65+), not just 25-45 year olds

AVOID THESE NAMES (already used): {existing_names_str}

For each person, provide:
- name: Full name (realistic for their ethnicity)
- age: Integer (realistic spread including 18-24 and 65+, not just 25-45)
- gender: "male", "female", or "non-binary"
- relationship: Role in household ("head", "spouse", "partner", "child", "parent", "flatmate")
- occupation: Current job or status (student, retired, unemployed, etc.)
- industry: Industry sector (e.g., "Healthcare", "Education", "Retail", "Trades", "Hospitality", "Technology", "Transport", "Finance", "Creative", "Public sector", "Retired", "Student", "Unemployed")
- education: Highest education level (e.g., "Secondary school", "GCSEs", "A-levels", "Vocational/trade certificate", "Some college", "Bachelor's degree", "Master's degree", "PhD", "No formal qualifications")
- ethnicity: Specific ethnicity
- political_lean: Political orientation (be realistic - include conservatives, apolitical people, not just progressives)
- personality_sketch: 2-3 sentences about personality, interests, daily life

Return ONLY a JSON array with EXACTLY {target_size} person object(s). No explanation."""
    
    return prompt, target_size

print("Prompt builder defined (now includes industry and education fields).")

Prompt builder defined (now includes industry and education fields).


### Cost Estimation

Before running 42 LLM calls, estimate the total cost based on prompt sizes.

In [9]:
if not SKIP_GENERATION:
    # Estimate total cost before running
    print("COST ESTIMATION")
    print("=" * 50)

    # Build sample prompts to estimate token counts
    sample_estimates = []
    simulated_names = []

    for household in households:
        prompt, expected_size = build_household_generation_prompt(household, simulated_names)
        
        # Estimate ~150 tokens per person in response (name, age, occupation, personality sketch, etc.)
        estimated_output_tokens = expected_size * 150
        
        estimate = estimate_cost(
            prompt=prompt,
            estimated_completion_tokens=estimated_output_tokens,
        )
        sample_estimates.append(estimate)
        
        # Simulate name accumulation for accurate prompt length estimation
        simulated_names.extend([f"Person {i}" for i in range(expected_size)])

    total_prompt_tokens = sum(e.prompt_tokens for e in sample_estimates)
    total_completion_tokens = sum(e.completion_tokens for e in sample_estimates)
    total_cost = sum(e.cost for e in sample_estimates)

    print(f"\nTarget: {TARGET_POPULATION} people across {len(households)} households")
    print(f"\nEstimated tokens:")
    print(f"  Prompt tokens:     {total_prompt_tokens:,}")
    print(f"  Completion tokens: {total_completion_tokens:,} (estimated)")
    print(f"  Total tokens:      {total_prompt_tokens + total_completion_tokens:,}")
    print(f"\n  Estimated cost: ${total_cost:.4f}")

    # Show cost breakdown
    print(f"\nPer-unit estimates:")
    print(f"  Per household: ${total_cost / len(households):.4f}")
    print(f"  Per person:    ${total_cost / TARGET_POPULATION:.4f}")
else:
    print("✓ Skipping cost estimation (using cached data)")

✓ Skipping cost estimation (using cached data)


In [10]:
# # Confirm before proceeding (comment out to skip)
# proceed = input(f"Proceed with generation? Estimated cost: ${total_cost:.4f} (y/n): ")
# if proceed.lower() != 'y':
#     raise KeyboardInterrupt("Generation cancelled by user")

In [11]:
import asyncio

if not SKIP_GENERATION:
    async def generate_household_members(household: Household, existing_names: list[str]) -> tuple[Household, list[dict], float]:
        """Generate members for a single household. Returns (household, members_data, cost)."""
        prompt, expected_size = build_household_generation_prompt(household, existing_names)
        
        result = await complete(prompt)
        
        text = result.content.strip()
        if text.startswith("```"): 
            text = text.split("```")[1].replace("json", "", 1).strip()
        
        try:
            members_data = json.loads(text)
        except json.JSONDecodeError as e:
            print(f"  ERROR parsing household {household.id}: {e}")
            members_data = []
        
        return household, members_data, result.cost, expected_size


    # Generate members for all households in parallel batches
    all_names = []
    total_generated = 0
    total_cost_actual = 0.0
    size_mismatches = []

    # Batch size - balance between parallelism and name deduplication
    BATCH_SIZE = 10

    print(f"Generating household members in batches of {BATCH_SIZE}...\n")

    for batch_start in tqdm(range(0, len(households), BATCH_SIZE), desc="Batches", unit="batch"):
        batch = households[batch_start:batch_start + BATCH_SIZE]
        
        # Run batch in parallel
        tasks = [generate_household_members(h, all_names.copy()) for h in batch]
        results = await asyncio.gather(*tasks)
        
        # Process results
        for household, members_data, cost, expected_size in results:
            total_cost_actual += cost
            
            if len(members_data) != expected_size:
                size_mismatches.append((household.id, expected_size, len(members_data)))
            
            for j, m in enumerate(members_data):
                member = HouseholdMember(
                    id=f"{household.id}_p{j+1}",
                    name=m["name"],
                    age=m["age"],
                    gender=m["gender"],
                    relationship=m["relationship"],
                    occupation=m["occupation"],
                    industry=m.get("industry", "Other"),  # New field
                    education=m.get("education", "Secondary school"),  # New field
                    ethnicity=m["ethnicity"],
                    political_lean=m["political_lean"],
                    personality_sketch=m["personality_sketch"],
                )
                household.members.append(member)
                all_names.append(m["name"])
            
            total_generated += len(members_data)

    print(f"\nComplete!")
    print(f"  Generated {total_generated} people across {len(households)} households")
    print(f"  Target was {TARGET_POPULATION} people")
    print(f"  Actual cost: ${total_cost_actual:.4f}")

    if size_mismatches:
        print(f"\n  WARNING: {len(size_mismatches)} households had size mismatches:")
        for hh_id, expected, actual in size_mismatches[:5]:
            print(f"    {hh_id}: expected {expected}, got {actual}")
        if len(size_mismatches) > 5:
            print(f"    ... and {len(size_mismatches) - 5} more")

    if total_generated != TARGET_POPULATION:
        print(f"\n  ⚠️  POPULATION MISMATCH: Got {total_generated}, expected {TARGET_POPULATION}")
    else:
        print(f"\n  ✓ Successfully generated exactly {TARGET_POPULATION} people")
else:
    print("✓ Skipping member generation (using cached data)")

✓ Skipping member generation (using cached data)


In [12]:
if not SKIP_GENERATION:
    # Preview some households (only after fresh generation)
    print("Sample households:\n")

    for household in households[:5]:
        print(f"{household.address}")
        print(f"  Type: {household.household_type} | Tenure: {household.tenure} | Years: {household.years_in_area}")
        for m in household.members:
            print(f"  - {m.name}, {m.age}, {m.gender} ({m.relationship})")
            print(f"    {m.occupation} | {m.industry} | {m.education}")
            print(f"    {m.ethnicity} | {m.political_lean}")
        print()
else:
    print("✓ Skipping preview (will show after loading cache in Stage 4)")

✓ Skipping preview (will show after loading cache in Stage 4)


---
## Stage 4: Validation

Check the generated neighbourhood for issues:
- Duplicate names
- Demographic distribution matches targets
- Household compositions are coherent

In [13]:
from collections import Counter

if SKIP_GENERATION:
    # Load from cached files
    print("Loading neighbourhood from cache...")
    
    with open(NEIGHBOURHOOD_FILE) as f:
        cached_data = json.load(f)
    
    # Reconstruct households and members from cached data
    households = []
    for hh_data in cached_data["households"]:
        addr_data = hh_data["address"]
        address = Address(
            street=addr_data["street"],
            number=addr_data["number"],
            postcode=addr_data["postcode"],
            property_type=addr_data["property_type"],
        )
        
        household = Household(
            id=hh_data["id"],
            address=address,
            household_type=hh_data["household_type"],
            tenure=hh_data["tenure"],
            years_in_area=hh_data["years_in_area"],
            socioeconomic=hh_data["socioeconomic"],
            members=[]
        )
        
        for m_data in hh_data["members"]:
            member = HouseholdMember(
                id=m_data["id"],
                name=m_data["name"],
                age=m_data["age"],
                gender=m_data["gender"],
                relationship=m_data["relationship"],
                occupation=m_data["occupation"],
                industry=m_data.get("industry", "Other"),  # Handle old data
                education=m_data.get("education", "Secondary school"),  # Handle old data
                ethnicity=m_data["ethnicity"],
                political_lean=m_data["political_lean"],
                personality_sketch=m_data["personality_sketch"],
            )
            household.members.append(member)
        
        households.append(household)
    
    print(f"  Loaded {len(households)} households from cache")

# Create neighbourhood object (works for both cached and newly generated)
neighbourhood = Neighbourhood(
    name="Dalston CLT",
    location="Dalston, London E8",
    households=households
)

all_members = neighbourhood.all_members()

print(f"\nNEIGHBOURHOOD SUMMARY")
print(f"="*50)
print(f"Total households: {len(neighbourhood.households)}")
print(f"Total residents: {neighbourhood.total_residents}")
print(f"Target population: {TARGET_POPULATION}")
print(f"Average household size: {neighbourhood.total_residents / len(neighbourhood.households):.1f}")

# Validate population
if neighbourhood.total_residents == TARGET_POPULATION:
    print(f"\n✓ Population target met: exactly {TARGET_POPULATION} people")
else:
    print(f"\n⚠️  Population mismatch: {neighbourhood.total_residents} vs target {TARGET_POPULATION}")

# Check for duplicate names
name_counts = Counter(m.name for m in all_members)
duplicates = {name: count for name, count in name_counts.items() if count > 1}

print(f"\nDUPLICATE NAMES: {len(duplicates)}")
if duplicates:
    for name, count in duplicates.items():
        print(f"  {name}: {count} occurrences")

Loading neighbourhood from cache...
  Loaded 41 households from cache

NEIGHBOURHOOD SUMMARY
Total households: 41
Total residents: 100
Target population: 100
Average household size: 2.4

✓ Population target met: exactly 100 people

DUPLICATE NAMES: 0


In [14]:
# Comprehensive demographic analysis
print("DEMOGRAPHIC ANALYSIS")
print("=" * 50)

# Helper to compare actual vs target
def compare_distribution(name: str, actual: dict, target: dict = None):
    print(f"\n{name}:")
    total = sum(actual.values())
    for key in sorted(actual.keys()):
        count = actual[key]
        pct = count / total * 100 if total > 0 else 0
        target_pct = target.get(key, 0) * 100 if target else None
        if target_pct is not None:
            diff = pct - target_pct
            flag = "⚠️" if abs(diff) > 10 else "✓" if abs(diff) < 5 else "~"
            print(f"  {key}: {count} ({pct:.1f}%) [target: {target_pct:.0f}%] {flag}")
        else:
            print(f"  {key}: {count} ({pct:.1f}%)")

# Age distribution (excluding children from target comparison)
adults = [m for m in all_members if m.age >= 18]
children = [m for m in all_members if m.age < 18]
print(f"\nTotal: {len(all_members)} people ({len(adults)} adults, {len(children)} children)")

age_brackets = {"18-24": 0, "25-34": 0, "35-44": 0, "45-54": 0, "55-64": 0, "65+": 0}
for m in adults:
    if m.age <= 24: age_brackets["18-24"] += 1
    elif m.age <= 34: age_brackets["25-34"] += 1
    elif m.age <= 44: age_brackets["35-44"] += 1
    elif m.age <= 54: age_brackets["45-54"] += 1
    elif m.age <= 64: age_brackets["55-64"] += 1
    else: age_brackets["65+"] += 1

compare_distribution("Age distribution (adults only)", age_brackets, DEMOGRAPHICS["age_brackets"])

# Gender distribution
gender_counts = Counter(m.gender.lower() for m in all_members)
compare_distribution("Gender distribution", dict(gender_counts))

# Ethnicity distribution
ethnicity_counts = Counter(m.ethnicity.lower() for m in all_members)
# Normalize ethnicity labels for comparison
ethnicity_normalized = {}
for eth, count in ethnicity_counts.items():
    # Try to map to our categories
    if "white" in eth and "british" in eth:
        key = "white_british"
    elif "white" in eth or "european" in eth or "irish" in eth or "polish" in eth or "italian" in eth:
        key = "white_other"
    elif "caribbean" in eth:
        key = "black_british_caribbean"
    elif "african" in eth or "nigerian" in eth or "ghanaian" in eth:
        key = "black_british_african"
    elif "turkish" in eth or "kurdish" in eth:
        key = "turkish_kurdish"
    elif "indian" in eth or "pakistani" in eth or "bangladeshi" in eth or "south asian" in eth:
        key = "south_asian"
    elif "chinese" in eth or "japanese" in eth or "korean" in eth or "east asian" in eth or "vietnamese" in eth:
        key = "east_asian"
    elif "mixed" in eth:
        key = "mixed"
    else:
        key = eth  # Keep original if no match
    ethnicity_normalized[key] = ethnicity_normalized.get(key, 0) + count

compare_distribution("Ethnicity distribution", ethnicity_normalized, DEMOGRAPHICS["ethnicity"])

# Political lean
political_counts = Counter(m.political_lean.lower() for m in all_members)
print(f"\nPolitical lean:")
for lean, count in sorted(political_counts.items(), key=lambda x: -x[1]):
    print(f"  {lean}: {count} ({count/len(all_members)*100:.1f}%)")

# Occupation diversity
occupation_counts = Counter(m.occupation.lower() for m in all_members)
print(f"\nOccupation diversity: {len(occupation_counts)} unique occupations")
print("Top 10:")
for occ, count in occupation_counts.most_common(10):
    print(f"  {occ}: {count}")

# Household-level stats
print(f"\n" + "=" * 50)
print("HOUSEHOLD-LEVEL ANALYSIS")

# Socioeconomic by tenure
print("\nSocioeconomic by tenure:")
for tenure in ["owner_occupied", "private_rented", "social_housing"]:
    hh_with_tenure = [h for h in households if h.tenure == tenure]
    if hh_with_tenure:
        socio_counts = Counter(h.socioeconomic for h in hh_with_tenure)
        print(f"  {tenure}:")
        for socio, count in sorted(socio_counts.items()):
            print(f"    {socio}: {count}")

# Years in area distribution
years_brackets = {"0-2": 0, "3-5": 0, "6-10": 0, "11-20": 0, "20+": 0}
for h in households:
    if h.years_in_area <= 2: years_brackets["0-2"] += 1
    elif h.years_in_area <= 5: years_brackets["3-5"] += 1
    elif h.years_in_area <= 10: years_brackets["6-10"] += 1
    elif h.years_in_area <= 20: years_brackets["11-20"] += 1
    else: years_brackets["20+"] += 1

compare_distribution("Years in area (households)", years_brackets, DEMOGRAPHICS["years_in_area"])

DEMOGRAPHIC ANALYSIS

Total: 100 people (94 adults, 6 children)

Age distribution (adults only):
  18-24: 25 (26.6%) [target: 12%] ⚠️
  25-34: 16 (17.0%) [target: 28%] ⚠️
  35-44: 17 (18.1%) [target: 22%] ✓
  45-54: 10 (10.6%) [target: 15%] ✓
  55-64: 3 (3.2%) [target: 12%] ~
  65+: 23 (24.5%) [target: 11%] ⚠️

Gender distribution:
  female: 56 (56.0%)
  male: 43 (43.0%)
  non-binary: 1 (1.0%)

Ethnicity distribution:
  black british: 14 (14.0%) [target: 0%] ⚠️
  east_asian: 3 (3.0%) [target: 5%] ✓
  greek: 1 (1.0%) [target: 0%] ✓
  south_asian: 9 (9.0%) [target: 8%] ✓
  turkish_kurdish: 32 (32.0%) [target: 12%] ⚠️
  white_british: 25 (25.0%) [target: 30%] ~
  white_other: 16 (16.0%) [target: 15%] ✓

Political lean:
  progressive: 21 (21.0%)
  liberal: 17 (17.0%)
  apolitical: 15 (15.0%)
  centrist: 10 (10.0%)
  moderate: 9 (9.0%)
  conservative: 8 (8.0%)
  labour: 4 (4.0%)
  centre-left: 3 (3.0%)
  neutral: 1 (1.0%)
  left-leaning: 1 (1.0%)
  traditional labour: 1 (1.0%)
  moderate li

---
## Stage 5: Export

Save the neighbourhood data for use in the Testing space before it happens experiment.

In [15]:
# Export to JSON (only if newly generated)
output_dir = Path('../data/synthetic/dalston_clt')
output_dir.mkdir(parents=True, exist_ok=True)

if not SKIP_GENERATION:
    # Convert to serializable format
    def household_to_dict(h: Household) -> dict:
        return {
            "id": h.id,
            "address": {
                "street": h.address.street,
                "number": h.address.number,
                "postcode": h.address.postcode,
                "property_type": h.address.property_type,
            },
            "household_type": h.household_type,
            "tenure": h.tenure,
            "years_in_area": h.years_in_area,
            "socioeconomic": h.socioeconomic,
            "members": [asdict(m) for m in h.members]
        }

    neighbourhood_data = {
        "name": neighbourhood.name,
        "location": neighbourhood.location,
        "generated_at": str(Path.cwd()),
        "total_households": len(neighbourhood.households),
        "total_residents": neighbourhood.total_residents,
        "households": [household_to_dict(h) for h in neighbourhood.households]
    }

    output_file = output_dir / 'neighbourhood.json'
    with open(output_file, 'w') as f:
        json.dump(neighbourhood_data, f, indent=2)

    print(f"Exported neighbourhood to {output_file}")
    print(f"  {neighbourhood_data['total_households']} households")
    print(f"  {neighbourhood_data['total_residents']} residents")
else:
    print(f"✓ Skipping export (using cached {NEIGHBOURHOOD_FILE.name})")

✓ Skipping export (using cached neighbourhood.json)


In [16]:
if not SKIP_GENERATION:
    # Export a flat list of personas for easy surveying
    # Now includes industry and education for direct SyntheticIdentity conversion
    personas_list = []

    for household in neighbourhood.households:
        for member in household.members:
            persona_data = {
                "id": member.id,
                "name": member.name,
                "age": member.age,
                "gender": member.gender,
                "occupation": member.occupation,
                "industry": member.industry,  # New field
                "education": member.education,  # New field
                "ethnicity": member.ethnicity,
                "political_lean": member.political_lean,
                "personality_sketch": member.personality_sketch,
                "household_id": household.id,
                "address": str(household.address),
                "tenure": household.tenure,
                "years_in_area": household.years_in_area,
                "socioeconomic": household.socioeconomic,
            }
            personas_list.append(persona_data)

    personas_file = output_dir / 'personas.json'
    with open(personas_file, 'w') as f:
        json.dump(personas_list, f, indent=2)

    print(f"Exported {len(personas_list)} personas to {personas_file}")
    print(f"  (includes industry and education for SyntheticIdentity compatibility)")
else:
    print(f"✓ Skipping personas export (using cached {PERSONAS_FILE.name})")

✓ Skipping personas export (using cached personas.json)


---
## Stage 6: Generate Data Files

Generate synthetic personal data files (CV, reading list, subscriptions) for each persona. These files will then be processed to create rich context statements.

In [17]:
# Cost estimation for file generation
# 
# For each persona, we generate 2-5 files based on their characteristics
# (not everyone has a CV - retirees, students, etc.)
# Then process them to extract a context statement

from centuria.persona import (
    SyntheticIdentity, 
    generate_synthetic_files, 
    list_available_file_types,
    infer_file_types_for_identity,
)
import os

# Load our personas
with open('../data/synthetic/dalston_clt/personas.json') as f:
    personas_data = json.load(f)

print(f"Loaded {len(personas_data)} personas")
print(f"\nAvailable file types: {list(list_available_file_types().keys())}")

# Estimate costs
print("\n" + "=" * 60)
print("COST ESTIMATION: File Generation + Context Extraction")
print("=" * 60)

# Per-file estimates (based on prompts in synthetic.py)
FILE_PROMPT_TOKENS = 350  # Average prompt size for file generation
FILE_COMPLETION_TOKENS = 250  # Average response size

# Extraction estimates (based on extractors.py)
EXTRACTION_PROMPT_TOKENS = 1800  # Profile extraction from files
EXTRACTION_COMPLETION_TOKENS = 200

# Context statement estimates  
CONTEXT_PROMPT_TOKENS = 1500  # Building context from profile + raw data
CONTEXT_COMPLETION_TOKENS = 500  # 300-500 word statement

# Average files per persona (using infer_file_types_for_identity)
AVG_FILES_PER_PERSONA = 3.5  # Range is 2-5, weighted average ~3.5

# Calculate per-persona totals
prompt_per_persona = (
    (FILE_PROMPT_TOKENS * AVG_FILES_PER_PERSONA) +
    EXTRACTION_PROMPT_TOKENS + 
    CONTEXT_PROMPT_TOKENS
)
completion_per_persona = (
    (FILE_COMPLETION_TOKENS * AVG_FILES_PER_PERSONA) +
    EXTRACTION_COMPLETION_TOKENS +
    CONTEXT_COMPLETION_TOKENS
)

# Total for all personas
num_personas = len(personas_data)
total_prompt_tokens = prompt_per_persona * num_personas
total_completion_tokens = completion_per_persona * num_personas

# Get model from environment
model = os.getenv("DEFAULT_MODEL", "gpt-4o")

# GPT-4o pricing (as of 2024): $2.50/1M input, $10/1M output
PRICE_PER_1M_INPUT = 2.50
PRICE_PER_1M_OUTPUT = 10.00

estimated_cost = (
    (total_prompt_tokens / 1_000_000) * PRICE_PER_1M_INPUT +
    (total_completion_tokens / 1_000_000) * PRICE_PER_1M_OUTPUT
)

print(f"\nConfiguration:")
print(f"  Model: {model}")
print(f"  Personas: {num_personas}")
print(f"  Files per persona: 2-5 (inferred based on occupation/age)")

print(f"\nPer-persona estimates (avg {AVG_FILES_PER_PERSONA} files):")
print(f"  Prompt tokens: {prompt_per_persona:,.0f}")
print(f"  Completion tokens: {completion_per_persona:,.0f}")
print(f"  Total tokens: {prompt_per_persona + completion_per_persona:,.0f}")

print(f"\nTotal estimates ({num_personas} personas):")
print(f"  Prompt tokens: {total_prompt_tokens:,.0f}")
print(f"  Completion tokens: {total_completion_tokens:,.0f}")  
print(f"  Total tokens: {total_prompt_tokens + total_completion_tokens:,.0f}")
print(f"\n  Estimated cost: ${estimated_cost:.2f} (GPT-4o pricing)")
print(f"  With 20% safety margin: ${estimated_cost * 1.2:.2f}")

Loaded 100 personas

Available file types: ['cv', 'linkedin_summary', 'work_calendar', 'email_signature', 'twitter_bio', 'instagram_bio', 'facebook_about', 'reddit_profile', 'tiktok_profile', 'nextdoor_activity', 'reading_list', 'subscriptions', 'spotify_favorites', 'netflix_history', 'youtube_subscriptions', 'podcast_subscriptions', 'amazon_wishlist', 'shopping_history', 'grocery_list', 'loyalty_programs', 'google_reviews', 'product_reviews', 'yelp_reviews', 'notes_snippet', 'bookmarks', 'text_messages', 'voicemail_greeting', 'fitness_tracker', 'health_goals', 'travel_history', 'location_history', 'bank_categories', 'charity_donations', 'home_description', 'pet_profile', 'vehicle_info', 'recipe_collection', 'dating_profile', 'forum_posts', 'event_attendance']

COST ESTIMATION: File Generation + Context Extraction

Configuration:
  Model: gpt-4o
  Personas: 100
  Files per persona: 2-5 (inferred based on occupation/age)

Per-persona estimates (avg 3.5 files):
  Prompt tokens: 4,525
  C

In [18]:
# Generate synthetic files for each persona
# Personas now include industry and education fields - direct SyntheticIdentity conversion

import asyncio
import random

def persona_to_identity(p: dict) -> SyntheticIdentity:
    """Convert persona dict to SyntheticIdentity - now a simple field mapping."""
    return SyntheticIdentity(
        name=p["name"],
        age=p["age"],
        gender=p["gender"],
        location=f"{p['address'].split(',')[0]}, Dalston, London",
        occupation=p["occupation"],
        industry=p["industry"],  # Direct from persona data
        education=p["education"],  # Direct from persona data
        political_lean=p["political_lean"],
        personality_sketch=p["personality_sketch"],
    )


# Preview file type inference for a few personas
print("File type inference preview:\n")
for p in personas_data[:5]:
    identity = persona_to_identity(p)
    num_files = random.randint(2, 5)
    inferred_types = infer_file_types_for_identity(identity, num_files=num_files)
    print(f"  {p['name']} ({p['age']}, {p['occupation']}, {p['industry']}):")
    print(f"    Education: {p['education']}")
    print(f"    -> Files: {inferred_types}")
print()

File type inference preview:

  Dilek Aydin (67, retired, Retired):
    Education: No formal qualifications
    -> Files: ['reading_list', 'shopping_history', 'spotify_favorites', 'charity_donations', 'bank_categories']
  Kemal Aydin (43, electrician, Trades):
    Education: Vocational/trade certificate
    -> Files: ['bookmarks', 'pet_profile']
  Zeynep Aydin (40, shop assistant, Retail):
    Education: GCSEs
    -> Files: ['bank_categories', 'cv', 'youtube_subscriptions', 'instagram_bio']
  Hasan Aydin (20, student, Student):
    Education: A-levels
    -> Files: ['yelp_reviews', 'spotify_favorites', 'location_history', 'youtube_subscriptions', 'reddit_profile']
  Emine Aydin (18, student, Student):
    Education: A-levels
    -> Files: ['travel_history', 'vehicle_info', 'spotify_favorites', 'home_description', 'shopping_history']



In [19]:
# Generate files for all personas
# CACHING: Skips personas that already have generated files
# File types are inferred per-person (2-5 files based on occupation/age)

PERSONAS_DIR = Path('../data/synthetic/dalston_clt/personas')

def persona_has_files(persona_id: str, output_base: Path) -> bool:
    """Check if persona already has generated files (at least identity.json + 1 other file)."""
    persona_dir = output_base / persona_id
    if not persona_dir.exists():
        return False
    has_identity = (persona_dir / "identity.json").exists()
    has_data_files = any(f.suffix == ".txt" for f in persona_dir.iterdir() if f.is_file())
    return has_identity and has_data_files


async def generate_files_for_persona(persona: dict, output_base: Path) -> tuple[str, dict, float]:
    """Generate files for a single persona. Returns (persona_id, saved_files, cost)."""
    identity = persona_to_identity(persona)
    
    # Infer appropriate file types for this person (2-5 files)
    num_files = random.randint(2, 5)
    file_types = infer_file_types_for_identity(identity, num_files=num_files)
    
    persona_dir = output_base / persona["id"]
    
    saved_files = await generate_synthetic_files(
        identity=identity,
        output_dir=persona_dir,
        file_types=file_types,
    )
    
    estimated_cost = len(saved_files) * 0.003
    return persona["id"], {k: str(v) for k, v in saved_files.items()}, estimated_cost


# Create output directory
PERSONAS_DIR.mkdir(parents=True, exist_ok=True)

# Check which personas need generation
personas_to_generate = [p for p in personas_data if not persona_has_files(p["id"], PERSONAS_DIR)]
personas_cached = [p["id"] for p in personas_data if persona_has_files(p["id"], PERSONAS_DIR)]

print(f"Persona file generation status:")
print(f"  Already cached: {len(personas_cached)}")
print(f"  Need generation: {len(personas_to_generate)}")

if not personas_to_generate:
    print(f"\n✓ All {len(personas_cached)} personas already have files. Skipping generation.")
    results = [{"persona_id": pid, "files": "cached"} for pid in personas_cached]
    total_cost = 0.0
    errors = []
else:
    print(f"\nGenerating files for {len(personas_to_generate)} personas concurrently...")
    print(f"Output directory: {PERSONAS_DIR}")
    print(f"Files per persona: 2-5 (inferred based on occupation/age)")
    print()

    results = [{"persona_id": pid, "files": "cached"} for pid in personas_cached]
    total_cost = 0.0
    errors = []
    file_type_counts = {}

    # Process ALL personas concurrently
    tasks = [generate_files_for_persona(p, PERSONAS_DIR) for p in personas_to_generate]
    all_results = await asyncio.gather(*tasks, return_exceptions=True)
    
    for i, result in enumerate(all_results):
        if isinstance(result, Exception):
            persona_id = personas_to_generate[i]["id"]
            errors.append((persona_id, str(result)))
            print(f"  ERROR: {persona_id}: {result}")
        else:
            persona_id, saved_files, cost = result
            results.append({"persona_id": persona_id, "files": saved_files})
            total_cost += cost
            for ft in saved_files.keys():
                file_type_counts[ft] = file_type_counts.get(ft, 0) + 1

    print(f"\n{'='*60}")
    print("FILE GENERATION COMPLETE")
    print(f"{'='*60}")
    print(f"  Cached (skipped): {len(personas_cached)}")
    print(f"  Newly generated: {len(personas_to_generate) - len(errors)}")
    print(f"  Errors: {len(errors)}")
    print(f"  Cost for new files: ${total_cost:.4f}")
    
    if file_type_counts:
        print(f"\n  File type distribution:")
        for ft, count in sorted(file_type_counts.items(), key=lambda x: -x[1]):
            print(f"    {ft}: {count}")

    if errors:
        print(f"\nErrors:")
        for persona_id, error in errors[:10]:
            print(f"  {persona_id}: {error}")
        if len(errors) > 10:
            print(f"  ... and {len(errors) - 10} more")

Persona file generation status:
  Already cached: 100
  Need generation: 0

✓ All 100 personas already have files. Skipping generation.


In [20]:
# Preview generated files for a sample persona
sample_persona_id = results[0]["persona_id"] if results else personas_data[0]["id"]
sample_dir = PERSONAS_DIR / sample_persona_id

print(f"Sample files for {sample_persona_id}:")
print(f"Directory: {sample_dir}")
print()

for file_path in sorted(sample_dir.iterdir()):
    if not file_path.is_file():  # Skip directories like .ipynb_checkpoints
        continue
    print(f"--- {file_path.name} ---")
    content = file_path.read_text()
    # Show first 500 chars
    if len(content) > 500:
        print(content[:500] + "\n...[truncated]")
    else:
        print(content)
    print()

Sample files for hh_001_p1:
Directory: ../data/synthetic/dalston_clt/personas/hh_001_p1

--- facebook_about.txt ---
**About/Intro:**
Hello, I'm Dilek! I'm a retired Londoner with a passion for gardening and cooking delicious Turkish meals for my loved ones. I enjoy being a part of my vibrant community here in Dalston and always love a good chat with friends and neighbors.

**Work and Education:**
- Occupation: Retired
- Education: No formal qualifications

**Relationship Status:**
- Married

**Places Lived:**
- Originally from Turkey
- Currently residing at 1 Hackett Green Road, Dalston, London

**Life Events
...[truncated]

--- identity.json ---
{
  "name": "Dilek Aydin",
  "age": 67,
  "gender": "female",
  "location": "1 Hackett Green Road, Dalston, London",
  "occupation": "retired",
  "industry": "Retired",
  "education": "No formal qualifications",
  "political_lean": "neutral",
  "personality_sketch": "Dilek enjoys tending to her small garden and cooking traditional Turkish meal

---
## Stage 7: Extract Rich Context Statements

Process the generated files through the extraction pipeline to create rich context statements for each persona. This uses the same `process_personal_folder` function used for real data.

In [21]:
# Cost estimation for context extraction
print("=" * 60)
print("COST ESTIMATION: Context Extraction")
print("=" * 60)

# Count personas with generated files
personas_with_files = list(PERSONAS_DIR.iterdir()) if PERSONAS_DIR.exists() else []
num_to_process = len(personas_with_files)

# Per-persona: profile extraction + context statement generation
# These are the expensive operations
context_prompt_tokens = EXTRACTION_PROMPT_TOKENS + CONTEXT_PROMPT_TOKENS  # ~3300
context_completion_tokens = EXTRACTION_COMPLETION_TOKENS + CONTEXT_COMPLETION_TOKENS  # ~700

total_context_prompt = context_prompt_tokens * num_to_process
total_context_completion = context_completion_tokens * num_to_process

# Calculate cost directly using token counts
context_cost = (
    (total_context_prompt / 1_000_000) * PRICE_PER_1M_INPUT +
    (total_context_completion / 1_000_000) * PRICE_PER_1M_OUTPUT
)

print(f"\nPersonas to process: {num_to_process}")
print(f"\nPer-persona estimates:")
print(f"  Prompt tokens: {context_prompt_tokens:,}")
print(f"  Completion tokens: {context_completion_tokens:,}")

print(f"\nTotal estimates:")
print(f"  Prompt tokens: {total_context_prompt:,}")
print(f"  Completion tokens: {total_context_completion:,}")
print(f"\n  💰 Estimated cost: ${context_cost:.2f}")

COST ESTIMATION: Context Extraction

Personas to process: 100

Per-persona estimates:
  Prompt tokens: 3,300
  Completion tokens: 700

Total estimates:
  Prompt tokens: 330,000
  Completion tokens: 70,000

  💰 Estimated cost: $1.53


In [22]:
# Extract context statements for all personas
# CACHING: Saves context statements to file and reuses them on rerun

# Force reload of the extractors module to pick up any changes
import importlib
import centuria.data.extractors
import centuria.data
importlib.reload(centuria.data.extractors)
importlib.reload(centuria.data)
from centuria.data.extractors import process_personal_folder

CONTEXT_CACHE_FILE = output_dir / 'context_cache.json'

# Load existing cache if available
if CONTEXT_CACHE_FILE.exists():
    with open(CONTEXT_CACHE_FILE) as f:
        context_cache = json.load(f)
    print(f"Loaded {len(context_cache)} cached context statements from {CONTEXT_CACHE_FILE.name}")
else:
    context_cache = {}
    print("No context cache found. Will generate all context statements.")


async def extract_context_for_persona(persona_id: str, persona_dir: Path) -> tuple[str, str, float]:
    """Extract context statement for a persona. Returns (persona_id, context_statement, cost)."""
    try:
        profile, context_statement = await process_personal_folder(str(persona_dir))
        estimated_cost = 0.01
        return persona_id, context_statement, estimated_cost
    except Exception as e:
        raise Exception(f"Failed to process {persona_id}: {e}")


# Get list of persona directories (filter out non-directories like .DS_Store)
persona_dirs = sorted([d for d in PERSONAS_DIR.iterdir() if d.is_dir()]) if PERSONAS_DIR.exists() else []

# Determine which need extraction
dirs_to_process = [d for d in persona_dirs if d.name not in context_cache]

print(f"\nContext extraction status:")
print(f"  Already cached: {len(persona_dirs) - len(dirs_to_process)}")
print(f"  Need extraction: {len(dirs_to_process)}")

# Start with cached results
context_results = dict(context_cache)
extraction_errors = []
extraction_cost = 0.0

if not dirs_to_process:
    print(f"\n✓ All {len(context_results)} context statements cached. Skipping extraction.")
else:
    print(f"\nExtracting context statements for {len(dirs_to_process)} personas concurrently...")
    
    # Process ALL personas concurrently
    tasks = [extract_context_for_persona(d.name, d) for d in dirs_to_process]
    all_results = await asyncio.gather(*tasks, return_exceptions=True)
    
    for i, result in enumerate(all_results):
        if isinstance(result, Exception):
            persona_id = dirs_to_process[i].name
            extraction_errors.append((persona_id, str(result)))
            print(f"  ERROR: {persona_id}: {result}")
        else:
            persona_id, context_statement, cost = result
            context_results[persona_id] = context_statement
            extraction_cost += cost

    # Save cache
    with open(CONTEXT_CACHE_FILE, 'w') as f:
        json.dump(context_results, f, indent=2)

    print(f"\n{'='*60}")
    print("CONTEXT EXTRACTION COMPLETE")
    print(f"{'='*60}")
    print(f"  Cached (skipped): {len(persona_dirs) - len(dirs_to_process)}")
    print(f"  Newly extracted: {len(dirs_to_process) - len(extraction_errors)}")
    print(f"  Errors: {len(extraction_errors)}")
    print(f"  Cost for new extractions: ${extraction_cost:.4f}")
    print(f"\n  Cache saved to: {CONTEXT_CACHE_FILE}")

    if extraction_errors:
        print(f"\nErrors:")
        for persona_id, error in extraction_errors[:10]:
            print(f"  {persona_id}: {error}")
        if len(extraction_errors) > 10:
            print(f"  ... and {len(extraction_errors) - 10} more")

num_to_process = len(context_results)

Loaded 36 cached context statements from context_cache.json

Context extraction status:
  Already cached: 36
  Need extraction: 64

Extracting context statements for 64 personas concurrently...
  ERROR: hh_024_p2: Failed to process hh_024_p2: 1 validation error for ExtractedProfile
years_experience
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='null', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing

CONTEXT EXTRACTION COMPLETE
  Cached (skipped): 36
  Newly extracted: 63
  Errors: 1
  Cost for new extractions: $0.6300

  Cache saved to: ../data/synthetic/dalston_clt/context_cache.json

Errors:
  hh_024_p2: Failed to process hh_024_p2: 1 validation error for ExtractedProfile
years_experience
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='null', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/

In [23]:
# Preview some context statements
print("Sample context statements:\n")

sample_ids = list(context_results.keys())[:3]
for persona_id in sample_ids:
    # Find persona name
    persona = next((p for p in personas_data if p["id"] == persona_id), None)
    name = persona["name"] if persona else persona_id
    
    print(f"{'='*60}")
    print(f"{name} ({persona_id})")
    print(f"{'='*60}")
    
    context = context_results[persona_id]
    # Show first 800 chars
    if len(context) > 800:
        print(context[:800] + "\n...[truncated]")
    else:
        print(context)
    print()

Sample context statements:

Kemal Aydin (hh_001_p2)
This individual resides in the United Kingdom and falls within the 30-35 age range. They are currently employed as an electrician in the electrical industry. Although their years of experience and educational background are unspecified, their professional activities are likely supported by key skills in electrical work, DIY, and fitness. This person appears to value practicality and organization in their work, as suggested by their analytical communication style and an independent work approach combined with a moderate risk tolerance. Continuous learning also seems to be a core value, as indicated by their active engagement in understanding their trade and advancements in related fields.

Their intellectual interests span engineering, technology, and history, which aligns with their consum
...[truncated]

Zeynep Aydin (hh_001_p3)
This individual’s profile presents limited information about their work and educational background. There 

---
## Stage 8: Export Final Personas

Combine the original persona data with the rich context statements and export for use in the Testing space before it happens experiment.

In [24]:
# Combine persona data with rich context statements
enriched_personas = []

for persona in personas_data:
    persona_id = persona["id"]
    
    # Get the rich context statement if available
    if persona_id in context_results:
        rich_context = context_results[persona_id]
    else:
        # Fallback to basic context from original data
        rich_context = f"{persona['personality_sketch']} Lives at {persona['address']}. Works as {persona['occupation']}. Political lean: {persona['political_lean']}."
    
    enriched_persona = {
        **persona,  # Keep all original fields
        "rich_context": rich_context,
        "has_synthetic_files": persona_id in context_results,
    }
    enriched_personas.append(enriched_persona)

# Count enriched vs fallback
enriched_count = sum(1 for p in enriched_personas if p["has_synthetic_files"])
fallback_count = len(enriched_personas) - enriched_count

print(f"Persona enrichment summary:")
print(f"  Total personas: {len(enriched_personas)}")
print(f"  With rich context: {enriched_count}")
print(f"  Fallback context: {fallback_count}")

Persona enrichment summary:
  Total personas: 100
  With rich context: 99
  Fallback context: 1


In [25]:
# Export enriched personas
output_file = output_dir / 'personas_enriched.json'

with open(output_file, 'w') as f:
    json.dump(enriched_personas, f, indent=2)

print(f"Exported {len(enriched_personas)} enriched personas to {output_file}")

# Also export a simplified version for the web experiment
# Just id, name, and rich_context
web_personas = [
    {
        "id": p["id"],
        "name": p["name"],
        "age": p["age"],
        "gender": p["gender"],
        "occupation": p["occupation"],
        "ethnicity": p["ethnicity"],
        "address": p["address"],
        "context": p["rich_context"],
    }
    for p in enriched_personas
]

web_output = output_dir / 'personas_for_survey.json'
with open(web_output, 'w') as f:
    json.dump(web_personas, f, indent=2)

print(f"Exported {len(web_personas)} web-ready personas to {web_output}")

Exported 100 enriched personas to ../data/synthetic/dalston_clt/personas_enriched.json
Exported 100 web-ready personas to ../data/synthetic/dalston_clt/personas_for_survey.json


In [26]:
# Final summary and statistics
print("=" * 60)
print("PIPELINE COMPLETE")
print("=" * 60)

# Context statement length statistics
context_lengths = [len(p["rich_context"]) for p in enriched_personas]
avg_length = sum(context_lengths) / len(context_lengths)
min_length = min(context_lengths)
max_length = max(context_lengths)

print(f"\nGenerated files:")
print(f"  {output_dir / 'personas_enriched.json'}")
print(f"  {output_dir / 'personas_for_survey.json'}")
print(f"  {PERSONAS_DIR}/ (individual persona folders)")

print(f"\nContext statement statistics:")
print(f"  Average length: {avg_length:.0f} characters")
print(f"  Min length: {min_length} characters")
print(f"  Max length: {max_length} characters")

print(f"\nSample persona for survey use:")
sample = enriched_personas[0]
print(f"  Name: {sample['name']}")
print(f"  Age: {sample['age']}")
print(f"  Occupation: {sample['occupation']}")
print(f"  Context preview: {sample['rich_context'][:200]}...")

PIPELINE COMPLETE

Generated files:
  ../data/synthetic/dalston_clt/personas_enriched.json
  ../data/synthetic/dalston_clt/personas_for_survey.json
  ../data/synthetic/dalston_clt/personas/ (individual persona folders)

Context statement statistics:
  Average length: 2729 characters
  Min length: 459 characters
  Max length: 3136 characters

Sample persona for survey use:
  Name: Dilek Aydin
  Age: 67
  Occupation: retired
  Context preview: Dilek is a retired individual residing in Dalston, London. She originally hails from Turkey and moved to London in 1985. Dilek has two children, born in 1980 and 1983, and she has been married since 1...


---
## Usage Example

How to use these personas in a survey (e.g., for Testing space before it happens):

In [ ]:
# Example: Create Persona objects for surveying
from centuria.models import Persona, Question
from centuria.persona import create_persona
from centuria.survey import ask_question

# Load the enriched personas
with open('../data/synthetic/dalston_clt/personas_for_survey.json') as f:
    survey_personas_data = json.load(f)

# Create Persona objects
survey_personas = [
    create_persona(
        name=p['name'],
        context=p['context'],
        persona_id=p['id']
    )
    for p in survey_personas_data
]

print(f"Created {len(survey_personas)} Persona objects for surveying")

# Example question for the Testing space before it happens experiment
garden_question = Question(
    id="garden_priority",
    text="A 0.3-acre plot of land near your home has become available for community use. What would be your top priority for how this space should be used?",
    question_type="single_select",
    options=[
        "Community garden for growing food",
        "Green space with trees and benches",
        "Children's playground",
        "Wildlife habitat / nature reserve",
        "Community events space",
        "Allotments for residents",
    ]
)

# Test with first 3 personas
print("\nSample responses to garden question:\n")
for persona in survey_personas[:3]:
    response = await ask_question(persona, garden_question)
    print(f"{persona.name}: {response.response}")